In [4]:
import cv2
import os
import numpy as np
import pandas as pd

def preprocess_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.medianBlur(mask, 5)
    _, thresholded = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    thresholded = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel, iterations=2)
    return thresholded

def count_wounds(mask_path):
    thresholded = preprocess_mask(mask_path)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)
    return num_contours

def locate_wounds(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    thresholded = preprocess_mask(mask_path)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    height, width = mask.shape[:2]
    third_x, third_y = width // 3, height // 3
    half_x, half_y = width // 2, height // 2
    regions = {
        "top-left": (0, 0, third_x, third_y),
        "top-middle": (third_x, 0, 2 * third_x, third_y),
        "top-right": (2 * third_x, 0, width, third_y),
        "center-left": (0, third_y, third_x, 2 * third_y),
        "center": (third_x, third_y, 2 * third_x, 2 * third_y),
        "center-right": (2 * third_x, third_y, width, 2 * third_y),
        "bottom-left": (0, 2 * third_y, third_x, height),
        "bottom-middle": (third_x, 2 * third_y, 2 * third_x, height),
        "bottom-right": (2 * third_x, 2 * third_y, width, height),
        "left": (0, 0, half_x, height),
        "right": (half_x, 0, width, height),
        "top": (0, 0, width, half_y),
        "bottom": (0, half_y, width, height),
    }

    locations = []
    
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] == 0:
            continue
        cx, cy = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])
        x, y, w, h = cv2.boundingRect(contour)
        bbox_area = w * h
        region_scores = {region: 0 for region in regions}
        for region, (x1, y1, x2, y2) in regions.items():
            overlap_x1, overlap_y1 = max(x1, x), max(y1, y)
            overlap_x2, overlap_y2 = min(x2, x + w), min(y2, y + h)
            overlap_width, overlap_height = max(0, overlap_x2 - overlap_x1), max(0, overlap_y2 - overlap_y1)
            overlap_area = overlap_width * overlap_height
            if overlap_area > 0:
                region_scores[region] = overlap_area / bbox_area
            if x1 <= cx <= x2 and y1 <= cy <= y2:
                region_scores[region] += 0.5
        max_score = max(region_scores.values())
        best_regions = [region for region, score in region_scores.items() if score == max_score]
        locations.append(best_regions)
    return locations

def analyze_wounds(mask_path):
    thresholded = preprocess_mask(mask_path)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    wound_details = []

    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter != 0:
            roundness = 4 * np.pi * area / (perimeter ** 2)
        else:
            roundness = 0 
        if roundness == 0:
            shape = "undefined"
        elif roundness > 0.7:
            shape = "round"
        elif roundness > 0.4 and roundness < 0.7:
            shape = "rough"      
        elif roundness < 0.4:
            shape = "flattened"

        size = "small" if area <= 50000 else ("medium" if area <= 200000 else "large")

        wound_details.append({"Area": area, "Perimeter": perimeter, "Roundness": roundness, "Shape": shape, "Size": size})

    return wound_details

def ordinal(n):
    if 10 <= n % 100 <= 20:
        suffix = "th"
    else:
        suffix = {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th")
    return f"{n}{suffix}"

def describe_image(mask_path):
    num_wounds = count_wounds(mask_path)
    locations = locate_wounds(mask_path)
    wound_details = analyze_wounds(mask_path)

    text = "In this image, "

    if num_wounds == 0:
        text += "no wounds are detected."
    else:
        text += f"{num_wounds} wounds are detected. "

        for i in range(num_wounds):
            wound_num = ordinal(i + 1)
            wound_location = locations[i][0]
            wound_size = wound_details[i]['Size']
            wound_shape = wound_details[i]['Shape']
            
            text += f"The {wound_num} wound is located in {wound_location}, with size {wound_size} and shape {wound_shape}. "

    return text

if __name__ == "__main__":
    # print(describe_image('./data/MosMedDataPlus/Test/GTs/Jun_coronacases_case8_233.png'))
    # Path to the folder containing the mask images
    folder_path = '/home/mcn/NgocThang_K66/effisegnet/Kvasir-SEG/validation/masks'
    data = []
    for filename in os.listdir(folder_path):
            mask_path = os.path.join(folder_path, filename)
            description = describe_image(mask_path)
            data.append({'Image': filename, 'Description': description})
    df = pd.DataFrame(data)
    xlsx_file_path = '/home/mcn/NgocThang_K66/effisegnet/Kvasir-SEG/validation/val.csv'
    df.to_csv(xlsx_file_path, index=False)
    print(df)

     Image                                        Description
0   27.jpg  In this image, 1 wounds are detected. The 1st ...
1   18.jpg  In this image, 1 wounds are detected. The 1st ...
2    0.jpg  In this image, 1 wounds are detected. The 1st ...
3    8.jpg  In this image, 1 wounds are detected. The 1st ...
4    7.jpg  In this image, 1 wounds are detected. The 1st ...
..     ...                                                ...
95  50.jpg  In this image, 3 wounds are detected. The 1st ...
96   3.jpg  In this image, 1 wounds are detected. The 1st ...
97  85.jpg  In this image, 1 wounds are detected. The 1st ...
98  11.jpg  In this image, 1 wounds are detected. The 1st ...
99  36.jpg  In this image, 1 wounds are detected. The 1st ...

[100 rows x 2 columns]
